In [1]:
import spotipy
import json
import webbrowser
import urllib.request
import spotipy.util as util
import pygeohash as pgh

In [2]:
with open("geocode_keys.txt", "r") as geokey_file:
    geo_key = geokey_file.read()

In [3]:
address1 = input("Enter starting address or postcode:")
address2 = input("Enter ending address or postcode:")

addresses = [address1, address2]
new_addresses = [address
                    .replace(",", "")
                    .replace(" ", "+") for address in addresses]

# print(new_addresses)

hash_list = []

for new_address in new_addresses:
    map_url = f"https://geocode.maps.co/search?q={new_address}&api_key={geo_key}"

    map_request = urllib.request.Request(map_url)
    map_response = urllib.request.urlopen(map_request)

    map_data = json.loads(map_response.read())
    lat = map_data[0]['lat']
    lon = map_data[0]['lon']

    # lat_list.append(lat)
    # lon_list.append(lon)

    hash_list.append(pgh.encode(float(lat), float(lon), precision=5))

print(hash_list)


['gcpvn', 'gcpvk']


In [4]:
with open("ticketmaster_keys.txt", "r") as key_file:
    ticketmaster_key = key_file.read()

with open("genreIDs.json", "r") as genreID_file:
    genre_list = json.loads(genreID_file.read())

In [10]:
geohash = "gcpuv"
unit = "km"
radius = 10
user_genre = str(input("Enter a genre IN LOWERCASE (choose from Rock, Pop, Electronic, Classical, Rap, or Country):"))
genre = genre_list[user_genre]

events_both_places = {}

for hash in hash_list:
    url = f"https://app.ticketmaster.com/discovery/v2/events.json?size=20&radius={radius}&genreId={genre}&geoPoint={hash}&classificationName=music&apikey={ticketmaster_key}"

    tm_request = urllib.request.Request(url)
    tm_response = urllib.request.urlopen(tm_request)

events_both_places = json.loads(tm_response.read())
event_names = events_both_places['_embedded']['events']

print(events_both_places)


{'_embedded': {'events': [{'name': 'Beach Weather', 'type': 'event', 'id': '17uYvOG62jmFg3C', 'test': False, 'url': 'https://www.ticketmaster.co.uk/beach-weather-london-12-11-2024/event/370060F72E23548E', 'locale': 'en-us', 'images': [{'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/31c/3dac73bd-85e7-47d7-b459-0b8b727f531c_TABLET_LANDSCAPE_LARGE_16_9.jpg', 'width': 2048, 'height': 1152, 'fallback': False}, {'ratio': '4_3', 'url': 'https://s1.ticketm.net/dam/a/3d0/5a9c42e1-0913-4528-8c05-63af1da453d0_CUSTOM.jpg', 'width': 305, 'height': 225, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/3d0/5a9c42e1-0913-4528-8c05-63af1da453d0_RECOMENDATION_16_9.jpg', 'width': 100, 'height': 56, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/3d0/5a9c42e1-0913-4528-8c05-63af1da453d0_RETINA_PORTRAIT_16_9.jpg', 'width': 640, 'height': 360, 'fallback': False}, {'ratio': '16_9', 'url': 'https://s1.ticketm.net/dam/a/31c/3dac73bd-85e7-47d7-b459-0b8

In [11]:
# if len(event_names) < 5:
#     radius*2
#     full_event_listings = json.loads(tm_response.read())


artist_names = [artist['name'] for artist in event_names]
artist_names_no_repeats = list(dict.fromkeys(artist_names))
print(artist_names_no_repeats)

['Beach Weather', 'BON ENTENDEUR', 'Holy Youth Movement', "L'Imperatrice", "Fat Freddy's Drop", 'Polo & Pan', 'Ben Böhmer', 'Justice', 'Maribou State', 'Todd Terry', 'Soul City: House Music Every Saturday Night', 'Orchestre Tout Puissant Marcel Duchamp - Restaurant Tables', 'Orchestre Tout Puissant Marcel Duchamp']


In [12]:
cred = "spotify_keys.json"
with open(cred, "r") as spotify_key_file:
    api_tokens = json.load(spotify_key_file)

spotify_key_file.close()

client_id = api_tokens['client_id']
client_secret = api_tokens['client_secret']
redirectURI = api_tokens['redirect']
username = api_tokens['username']

scope = 'user-read-private user-read-playback-state user-modify-playback-state playlist-modify-public user-library-read'
token = util.prompt_for_user_token(username, scope, client_id=client_id,
                           client_secret=client_secret,
                           redirect_uri=redirectURI)

In [13]:
sp = spotipy.Spotify(auth=token)
# print(genre_list)

playlist_songs = []

for artist in artist_names_no_repeats:
    returned_song = sp.search(q=artist, type="track", limit=1)

    if returned_song['tracks']['items'][0]['uri'] not in playlist_songs:
        playlist_songs.append(returned_song['tracks']['items'][0]['uri'])

genre_name = list(genre_list.keys())[list(genre_list.values()).index(genre)]
# print(genre_name)

empty_playlist = sp.user_playlist_create(user=username, name=f"Journey from {address1.upper()} to {address2.upper()}", public=True, description=f"{genre_name} artists who've played recently anywhere within a {radius} {unit} radius of {address1.upper()} and {address2.upper()}")
full_playlist = sp.user_playlist_add_tracks(username, empty_playlist['id'], playlist_songs)

In [14]:
webbrowser.open(empty_playlist['external_urls']['spotify'])

True